In [107]:
import pandas as pd
import re, subprocess, pyodbc 
import numpy as np # linear algebra 
from sqlalchemy import func, create_engine
from config import mongoConn, connstr  
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
sw=stopwords.words('english')
ps=PorterStemmer()
import json 

In [186]:
engine = create_engine(connstr)
conn = engine.connect()
query = """ \
	SELECT ID, QGROUP, QPK, MAX_Q_PK, QOFFSET, MAX_QG_PK, QGOFFSET, MAX_FK_Form,QTEXT, IdText
	FROM vw_MetricsCompositeKeys WHERE PK_Form = '2020-A-SAOP'   
""" 
df = pd.read_sql(query,con=conn)   
df.to_excel(r'C:\_som\_src\_compile\saop\SAOP.xlsx','SAOP')  

In [187]:
QGR = 3000
d = {} 
for qg in df.QGROUP.unique():
    QGR+=1
    d[str(qg)]= str(QGR)
d    

{'2150': '3001',
 '2151': '3002',
 '2152': '3003',
 '2153': '3004',
 '2154': '3005',
 '2155': '3006',
 '2156': '3007',
 '2157': '3008',
 '2158': '3009',
 '2159': '3010',
 '2160': '3011',
 '2161': '3012',
 '2162': '3013',
 '2163': '3014',
 '2164': '3015',
 '2165': '3016',
 '2166': '3017'}

In [188]:
QGOFFSET = -1
QPK = 24500 
for i,r in df.iterrows():      
    if QGOFFSET != r.QGOFFSET:
        QGOFFSET = r.QGOFFSET
        QPK += 25
    QPK+=1
    d[str(r.QPK)]= str(QPK)  

In [189]:
with open("C:\_som\_src\_compile\saop\gapkeyval.json", "w") as f: 
    json.dump(d, f, indent=4)

In [190]:
dfid = df
dfid.replace("", np.NaN, inplace=True) 
dfid.dropna(subset = ["IdText"], inplace=True)
dfid['QHASH'] = dfid.QTEXT.apply(encode)  

In [191]:
did = {}
for i,r in df.iterrows(): 
    did[f'--\s{r.IdText}\s\s'] = '--\s ' + r.IdText + ' IDT:{' + r.QHASH + '} '
with open("C:\_som\_src\_compile\saop\did.json", "w") as f: 
    json.dump(did, f, indent=4)
did    


{'--\\s1a\\s\\s': '--\\s 1a IDT:{HSHDGNCYDSGNTDSNRGNCYFFCLPRVCYSPRQRDXCTVRDR13719MBGDNC} ',
 '--\\s1b\\s\\s': '--\\s 1b IDT:{HSGNCYRPRTDNMTTLCNTCTNFRMTNCRRNTSPMBMXWBSTFDRLPRVCYCNCLNDCTNGNCYDSGNTDSP} ',
 '--\\s1c\\s\\s': '--\\s 1c IDT:{DSSPNCSSRYPSTNXPRTSTHRTYSRVRLSPSLCTPPLYNDCTNGNCYDSGNTDSP} ',
 '--\\s1d\\s\\s': '--\\s 1d IDT:{DSSPNCSSRYRLRSPNSBLTSWTHNGNCYFLLWNGSLCTPPLYNDCTNGNCYDSGNTDSP} ',
 '--\\s1e\\s\\s': '--\\s 1e IDT:{HSGNCYDVLPDMNTNDPRVCYPRGRMPLN} ',
 '--\\s1f\\s\\s': '--\\s 1f IDT:{DSGNCYPRVCYPRGRMPLNNCLDDSCRPTNFLLWNGSLCTPPLYNDCTNGNCYDVLPDMNTNDPRVCYPRGRMPLN} ',
 '--\\s2a\\s\\s': '--\\s 2a IDT:{DSGNCYMNTNNVNTRYGNCYNFRMTNSYSTMSCRTCLLCTSPRCSSSTRMNTNDSSMNTDSCLSDSPSPRSNLLYDNTFBLNFRMTNP} ',
 '--\\s2b\\s\\s': '--\\s 2b IDT:{NMBRNFRMTNSYSTMSRPRTDRSPNSQSTN11FY2020CHFNFRMTNFFCRFSMMTRCSSDCRTCLLCTSPRCSSSTRMNTNDSSMNTDSCLSDSPSP} ',
 '--\\s2c\\s\\s': '--\\s 2c IDT:{NMBRNFRMTNSYSTMSRPRTDQSTN2BGNCYTHRZDRTHRZDPRTRPRTNGPRD} ',
 '--\\s2d\\s\\s': '--\\s 2d IDT:{NMBRNFRMTNSYSTMSRPRTDQSTN2CSPRVWDPPRVD

In [60]:
conn.close()

In [166]:
t = """
Has the as Languages technologies platforms is your web don't we have web driver 
"""
def encode(t):
    t = re.sub('[^A-Za-z0-9]', ' ', t ) 
    t = ' '.join([w for w in t.split(' ') if w not in sw])
    #t = ' '.join([ps.stem(w) for w in t.split(' ')])
    t = t.upper()
    t = re.sub('[AEIOU\s]', '', t)  
    return t
encode(t)

'HSLNGGSTCHNLGSPLTFRMSWBWBDRVR'